In [1]:
import cupy as cp  # Importar CuPy
from qutip import *
import dask
from dask import delayed
import dask.bag as db
from dask.distributed import Client, LocalCluster, progress
import functions_fbq
from functions_fbq import *
from scipy.interpolate import CubicSpline

c:\Users\jc269556\Anaconda3\envs\gpu\lib\site-packages\qutip\__init__.py:96: UserWarning: matplotlib not found: Graphics will not work.
  warnings.warn("matplotlib not found: Graphics will not work.")


In [50]:
cluster = LocalCluster(n_workers=20, threads_per_worker=1)
client = Client(cluster)
print(client)

c:\Users\jc269556\Anaconda3\envs\gpu\lib\site-packages\distributed\node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57891 instead
  warnings.warn(


In [117]:
def eigensystem_and_matrix_elements_sqr_fbq(Ec,El,EDelta,phi_ext,r, N = 200, eigvals = 0):
    # Obtain the eigenvalues, eigenkets and the |<1|O|0>|^2 of the n operator, phi operator and dH/dphi_ext operator.
    phi_ZPF = (2.0 * Ec / El) ** 0.25
    div_factor = 1 / (float(phi_ZPF) * 2)
    mul_factor = float(phi_ZPF)
    
    N_op = Qobj(1j * (destroy(N).dag() - destroy(N)).data.toarray() * div_factor)  # Multiplicar directamente por div_factor
    phi_op = Qobj((destroy(N).dag() + destroy(N)).data.toarray() * mul_factor)

    phi_op_cupy = cp.array(phi_op.data.toarray())
    r_cupy = cp.array(r)

    delta = phi_op-phi_ext
    ReZ = cp.cos(phi_op_cupy / 2) * cp.cos(r_cupy * phi_op_cupy / 2) + r_cupy * cp.sin(phi_op_cupy / 2) * cp.sin(r_cupy * phi_op_cupy / 2)  # Re(Z) of the Hamiltonian
    ImZ = -cp.cos(phi_op_cupy / 2) * cp.sin(r_cupy * phi_op_cupy / 2) + r_cupy * cp.sin(phi_op_cupy / 2) * cp.cos(r_cupy * phi_op_cupy / 2)  # Im(Z) of the Hamiltonian

    H = 4*Ec*tensor(N_op,qeye(2))**2 + 0.5*El*tensor(delta,qeye(2))**2 + EDelta*(tensor(ReZ,sigmaz())+tensor(ImZ,sigmay())) #Hamiltonian.
    evals,ekets=H.eigenstates(eigvals=eigvals)
    evals = np.real(evals)

    dReZdr = 1/2*(r*phi_op*(r*phi_op/2).cosm()*(phi_op/2).sinm()+(-phi_op*(phi_op/2).cosm()+2*(phi_op/2).sinm())*(r*phi_op/2).sinm())
    dImZdr = -1/2*(r*phi_op/2).cosm()*(phi_op*(phi_op/2).cosm()-2*(phi_op/2).sinm())-1/2*r*phi_op*(phi_op/2).sinm()*(r*phi_op/2).sinm()
    dHdr = EDelta*(tensor(dReZdr,sigmaz())+tensor(dImZdr,sigmay()))

    N_op01 = tensor(N_op,qeye(2)).matrix_element(ekets[1],ekets[0]) 
    phi_op01 = tensor(phi_op,qeye(2)).matrix_element(ekets[1],ekets[0])
    dHdr_op01 = dHdr.matrix_element(ekets[1],ekets[0])

    matrix_op_sqr_list = np.array([N_op01,phi_op01,dHdr_op01])
    return evals,ekets,matrix_op_sqr_list

def eigenenergies_fbq(Ec,El,EDelta,phi_ext,r, N = 200, eigvals = 0):
    # ONLY EIGENVALUES OF THE FERMIONIC-BOSONIC QUBIT 

    phi_ZPF = (2.0 * Ec / El) ** 0.25
    div_factor = 1 / (float(phi_ZPF) * 2)
    mul_factor = float(phi_ZPF)
    
    N_op = Qobj(1j * (destroy(N).dag() - destroy(N)).data.toarray() * div_factor)  # Multiplicar directamente por div_factor
    phi_op = Qobj((destroy(N).dag() + destroy(N)).data.toarray() * mul_factor)

    phi_op_cupy = cp.array(phi_op.data.toarray())
    r_cupy = cp.array(r)
    
    delta = phi_op - phi_ext
    ReZ = cp.cos(phi_op_cupy / 2) * cp.cos(r_cupy * phi_op_cupy / 2) + r_cupy * cp.sin(phi_op_cupy / 2) * cp.sin(r_cupy * phi_op_cupy / 2)  # Re(Z) of the Hamiltonian
    ImZ = -cp.cos(phi_op_cupy / 2) * cp.sin(r_cupy * phi_op_cupy / 2) + r_cupy * cp.sin(phi_op_cupy / 2) * cp.cos(r_cupy * phi_op_cupy / 2)  # Im(Z) of the Hamiltonian

    H = 4*Ec*tensor(N_op**2,qeye(2)) + 0.5*El*tensor((delta)**2,qeye(2)) + EDelta*(tensor(ReZ,sigmaz())+tensor(ImZ,sigmay())) #Hamiltonian.
    evals=H.eigenenergies(eigvals=eigvals)

    return evals

In [118]:
Ec = 40e9
EDelta = 20e9
eigvals = 2
phi_ext = cp.pi  # Usar cp.pi en lugar de np.pi

El_list = cp.logspace(cp.log10(10e6), cp.log10(50e9), 30)  # Reemplazar np.logspace con cp.logspace
r_list = cp.linspace(0.01, 0.99, 30)  # Reemplazar np.linspace con cp.linspace
El_r_pairs = [(El, r) for El in El_list for r in r_list]

In [119]:
# Cambiar a CuPy en lugar de NumPy para las matrices
f01_list = cp.zeros((len(El_list), len(r_list)))
dHdphiext_over_Delta_sqr_list = cp.zeros((len(El_list), len(r_list)))
dHdnext_over_Delta_sqr_list = cp.zeros((len(El_list), len(r_list)))
dHdr_over_Delta_sqr_list = cp.zeros((len(El_list), len(r_list)))
d2E01dphi2_list = cp.zeros((len(El_list), len(r_list)))

In [120]:
def process_r(El, r):
    # Aquí es donde podrías encontrar problemas de compatibilidad entre CuPy y otras bibliotecas como qutip

    El = cp.array(El) if cp.ndim(El) > 0 else cp.array([El])
    Ec = cp.array([40e9] * len(El))
    EDelta = cp.array([20e9] * len(El))
    
    eigvals_list, _, matrix_op_sqr_list = eigensystem_and_matrix_elements_sqr_fbq(Ec, El, EDelta, phi_ext, r, N=100, eigvals=eigvals)
    f01 = cp.real(eigvals_list[1] - eigvals_list[0])  # Usar cp.real en lugar de np.real
    n_matrix_element = matrix_op_sqr_list[0]
    phi_matrix_element = matrix_op_sqr_list[1]
    dH_dr_element = matrix_op_sqr_list[2]

    phi_ext_list_lower = cp.linspace(phi_ext - 0.01, phi_ext, 5, endpoint=False)  # Reemplazar np.linspace con cp.linspace
    phi_ext_list_upper = cp.linspace(phi_ext, phi_ext + 0.01, 6, endpoint=True)[1:]  # Reemplazar np.linspace con cp.linspace
    eigvals_list_lower = cp.zeros(len(phi_ext_list_lower))  # Reemplazar np.zeros con cp.zeros
    eigvals_list_upper = cp.zeros(len(phi_ext_list_upper))  # Reemplazar np.zeros con cp.zeros
    for i, phi_ext_aux in enumerate(phi_ext_list_upper):
            evals = eigenenergies_fbq(Ec, El, EDelta, phi_ext_aux, r, N=100, eigvals=2)
            eigvals_list_upper[i] = cp.real(evals[1] - evals[0])  # Usar cp.real en lugar de np.real

    phi_ext_list = cp.concatenate([phi_ext_list_lower, [phi_ext], phi_ext_list_upper])  # Reemplazar np.concatenate con cp.concatenate
    eigvals_phi_ext_list = cp.concatenate([eigvals_list_lower, [f01], eigvals_list_upper])  # Reemplazar np.concatenate con cp.concatenate

    # La interpolación cúbica de SciPy no funciona directamente con CuPy, así que necesitas convertir a NumPy antes de usarla
    phi_ext_list_np = cp.asnumpy(phi_ext_list)
    eigvals_phi_ext_list_np = cp.asnumpy(eigvals_phi_ext_list)
    cubic_spline = CubicSpline(phi_ext_list_np, eigvals_phi_ext_list_np)

    second_derivative_spline = cubic_spline.derivative(2)
    second_derivative_x0 = second_derivative_spline(cp.asnumpy(phi_ext))  # Convertir phi_ext a NumPy antes de usarlo en el spline

    return f01, phi_matrix_element, n_matrix_element, dH_dr_element, second_derivative_x0


In [121]:
results_bag = db.from_sequence(El_r_pairs).map(lambda x: process_r(x[0], x[1]))

In [122]:
El_test, r_test = El_list[0], r_list[0]
result_test = process_r(El_test, r_test)
print(result_test)

TypeError: Unsupported type <class 'qutip.qobj.Qobj'>

In [17]:
# Calcular los resultados
computed_results = dask.compute(*results_bag)

TypeError: Incompatible object for division